# Chap5. Getting Started with Pandas
## Introduction to pandas Data Stuctures
## Essential Functionality
## Summarizing and Computing Descriptive Statistics
## Handling Missing Data
## Hierachical Indexing
## Other pandas Topics